In [1]:
import re

def parse_test_log(log_text):
    """
    输入一段日志文本(多行)，解析出一个 step_info 列表。
    每个元素是一个 dict，包含:
      {
        'step': int,
        'AoMA_0': float,
        'AoMA_1': float,
        'loc': int,
        'battery': float,
        'action': str,
        'done': bool,
        'anchor_fail': bool
      }
    """
    step_info_list = []

    # 用正则表达式来匹配各字段，也可以手写split
    pattern = re.compile(
        r"Step\s+(\d+):\s+AoMA_0=([\d\.]+),\s+AoMA_1=([\d\.]+),\s+loc=([\d\.]+),\s+battery=([\d\.]+)\s*\|\s*Action=(.*?)\s*=>\s*Reward=([\-\d\.]+),\s*Done=(\w+),\s*AnchorFail=(\w+)"
    )
    
    for line in log_text.splitlines():
        line = line.strip()
        if not line:
            continue
        m = pattern.search(line)
        if m:
            step_i = int(m.group(1))
            aoma0 = float(m.group(2))
            aoma1 = float(m.group(3))
            loc   = int(float(m.group(4)))  # loc=2.0 => int(2.0)->2
            batt  = float(m.group(5))
            action = m.group(6).strip()
            # reward = float(m.group(7))  # 如果想要用也可以解析
            done_str = m.group(8)
            anchor_str = m.group(9)
            
            done = True if done_str == "True" else False
            anchor_fail = True if anchor_str == "True" else False

            step_info_list.append({
                "step": step_i,
                "AoMA_0": aoma0,
                "AoMA_1": aoma1,
                "loc": loc,
                "battery": batt,
                "action": action,
                "done": done,
                "anchor_fail": anchor_fail,
            })
    return step_info_list


In [4]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def parse_test_log(log_text):
    """
    输入一段日志文本(多行)，解析出一个 step_info 列表。
    每个元素是一个 dict，包含:
      {
        'step': int,
        'AoMA_0': float,
        'AoMA_1': float,
        'loc': int,
        'battery': float,
        'action': str,
        'done': bool,
        'anchor_fail': bool
      }
    """
    pattern = re.compile(
        r"Step\s+(\d+):\s+AoMA_0=([\d\.]+),\s+AoMA_1=([\d\.]+),\s+loc=([\d\.]+),\s+battery=([\d\.]+)\s*\|\s*Action=(.*?)\s*=>\s*Reward=([\-\d\.]+),\s*Done=(\w+),\s*AnchorFail=(\w+)"
    )
    step_info_list = []
    for line in log_text.splitlines():
        line = line.strip()
        if not line:
            continue
        m = pattern.search(line)
        if m:
            step_i = int(m.group(1))
            aoma0 = float(m.group(2))
            aoma1 = float(m.group(3))
            loc   = int(float(m.group(4)))
            batt  = float(m.group(5))
            action = m.group(6).strip()
            done_str = m.group(8)
            anchor_str = m.group(9)

            done = (done_str == "True")
            anchor_fail = (anchor_str == "True")

            step_info_list.append({
                "step": step_i,
                "AoMA_0": aoma0,
                "AoMA_1": aoma1,
                "loc": loc,
                "battery": batt,
                "action": action,
                "done": done,
                "anchor_fail": anchor_fail,
            })
    return step_info_list

def create_plotly_animation(step_info_list):
    """
    使用Plotly构建动画:
      - 三个固定地点: L0(0,1), L1(0,-1), Home(2,0)
      - 一个散点表示机器人位置, 每一帧更新位置
      - 并在注释/文本中展示AoMA、电池、动作等
    返回一个 Figure对象, 调用 fig.show() 即可交互观看
    """

    # 三个地点的坐标
    loc_coords = {
        0: (0, 1),   # L0
        1: (0, -1),  # L1
        2: (2, 0),   # Home
    }

    # 将三地点 + 机器人放在同一个散点图上
    # 我们先创建 基础 trace: L0, L1, Home(静态) + Robot(初始位置)
    # 后面再用 frames 来更新 Robot 的位置和附加信息

    # 准备地点散点(静态)
    x_points = [loc_coords[0][0], loc_coords[1][0], loc_coords[2][0]]
    y_points = [loc_coords[0][1], loc_coords[1][1], loc_coords[2][1]]
    text_points = ['L0', 'L1', 'Home']

    # 机器人的初始位置: 取第一个step
    if len(step_info_list) == 0:
        # 没有任何step时,返回空图
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_points, y=y_points, mode='markers+text', text=text_points))
        return fig

    start_loc = step_info_list[0]["loc"]
    rx0, ry0 = loc_coords[start_loc]

    # 创建Figure
    fig = go.Figure()

    # 添加地点Trace (静态)
    fig.add_trace(go.Scatter(
        x=x_points, y=y_points,
        mode='markers+text',
        text=text_points,
        textposition='top center',
        marker=dict(color=['red','blue','green'], size=12),
        name='Locations'
    ))
    
    # 添加机器人Trace (动态更新)
    robot_scatter = go.Scatter(
        x=[rx0], y=[ry0],
        mode='markers',
        marker=dict(size=12, symbol='square', color='black'),
        name='Robot'
    )
    fig.add_trace(robot_scatter)

    # 下面要准备 frames, 每个frame对应step_info_list中一个step
    # 并更新: 机器人散点坐标, 以及显示文本

    frames = []
    for i, step_info in enumerate(step_info_list):
        loc = step_info["loc"]
        rx, ry = loc_coords[loc]
        aoma0 = step_info["AoMA_0"]
        aoma1 = step_info["AoMA_1"]
        batt  = step_info["battery"]
        action = step_info["action"]
        step_num = step_info["step"]

        # 可以把想展示的信息放进 layout 的 annotation 或者 trace 的 text属性里
        # 这里我们在 layout 的 annotations 中更新
        text_annotation = (
            f"<b>Step {step_num}</b><br>"
            f"AoMA_0={aoma0:.1f}, AoMA_1={aoma1:.1f}<br>"
            f"Battery={batt:.1f}<br>"
            f"Action={action}"
        )

        frame = go.Frame(
            data=[
                # 对应顺序: 第一个散点是Locations(不变), 第二个散点是Robot
                go.Scatter(x=x_points, y=y_points), 
                go.Scatter(x=[rx], y=[ry]),  
            ],
            name=f"frame_{i}",
            layout=go.Layout(
                annotations=[
                    dict(
                        x=0.5, y=1.1, xref="paper", yref="paper",
                        text=text_annotation,
                        showarrow=False,
                        font=dict(size=14, color="purple"),
                    )
                ]
            )
        )
        frames.append(frame)

    fig.frames = frames

    # 定义一个 slider, 让用户可以拖动查看每一帧
    sliders = [
        dict(
            steps=[
                dict(
                    method="animate",
                    args=[
                        [f"name == frame_{i}"],
                        dict(mode="immediate", frame=dict(duration=600, redraw=True), transition=dict(duration=0))
                    ],
                    label=f"{i}"
                ) for i in range(len(frames))
            ],
            transition=dict(duration=0),
            x=0.1, y=0,
            currentvalue=dict(font=dict(size=12), prefix="StepIndex: "),
            len=0.9
        )
    ]

    # 定义动画按钮（播放/暂停）
    updatemenus = [
        dict(
            type="buttons",
            showactive=False,
            x=0.05, y=-0.1,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,  # 表示播放全部frame
                        dict(frame=dict(duration=600, redraw=True), fromcurrent=True, mode='immediate')
                    ]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        dict(frame=dict(duration=0, redraw=False), mode='immediate')
                    ]
                ),
            ]
        )
    ]

    fig.update_layout(
        width=700, height=600,
        xaxis=dict(range=[-1, 3], autorange=False),
        yaxis=dict(range=[-2, 2], autorange=False),
        title="Agent Patrol Visualization (Plotly)",
        showlegend=False,
        updatemenus=updatemenus,
        sliders=sliders
    )

    # 初始状态：显示第一帧时的 annotation
    init_annotation = [
        dict(
            x=0.5, y=1.1, xref="paper", yref="paper",
            text=(
                f"<b>Step {step_info_list[0]['step']}</b><br>"
                f"AoMA_0={step_info_list[0]['AoMA_0']:.1f}, AoMA_1={step_info_list[0]['AoMA_1']:.1f}<br>"
                f"Battery={step_info_list[0]['battery']:.1f}<br>"
                f"Action={step_info_list[0]['action']}"
            ),
            showarrow=False,
            font=dict(size=14, color="purple"),
        )
    ]
    fig.update_layout(annotations=init_annotation)

    return fig


def demo_run():
    # 这里手动模拟一小段日志, 你可以换成你自己的日志文本
    # 读取 test1.txt 文件内容
    with open("test1.txt", "r", encoding="utf-8") as f:
        log_text = f.read()  # 读取整个文件内容作为字符串

    # 1. 解析日志
    steps = parse_test_log(log_text)
    # 2. 创建plotly动画
    fig = create_plotly_animation(steps)
    # 3. 展示
    fig.show()


# 如果你直接运行这个文件, 就执行demo_run
if __name__ == "__main__":
    demo_run()
